In [1]:
import sys
import requests
sys.path.append("..")  # Add the parent directory to the sys.path
from config import CR_EMAIL, API_KEY, BASE_URL

/Users/kethan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
def get_total_unique_spaces_and_pages_in_confluence():
    email = CR_EMAIL
    api_token = API_KEY
    auth = (email, api_token)
    base_url = "https://cybereason.atlassian.net/wiki/rest/api/"
    unique_spaces = set()
    total_unique_pages = 0

    # Make request to Confluence API to get total number of spaces
    limit_spaces = 10000  # Adjust the limit as needed
    start_spaces = 0

    while True:
        spaces_response = requests.get(base_url + f"space?start={start_spaces}&limit={limit_spaces}", auth=auth)
        spaces_data = spaces_response.json()
        current_spaces = spaces_data.get('results', [])

        if not current_spaces:
            break

        # Loop through each space
        for space in current_spaces:
            space_key = space['key']
            space_name = space['name']
            unique_spaces.add(space_key)

            # Reset total_unique_pages counter for each space
            total_unique_pages_space = 0

            # Make request to Confluence API to get total number of pages for current space
            pages_response = requests.get(base_url + f"content?spaceKey={space_key}&limit=1", auth=auth)
            total_pages = pages_response.json().get('size', 0)

            # Initialize variables for pagination
            limit_pages = 10000  # Adjust the limit as needed
            start_pages = 0

            while start_pages < total_pages:
                # Make request to Confluence API to fetch pages for current space with pagination
                pages_response = requests.get(base_url + f"content?spaceKey={space_key}&limit={limit_pages}&start={start_pages}", auth=auth)
                pages_data = pages_response.json().get('results', [])

                # Count unique pages for current space
                unique_pages = set(page['id'] for page in pages_data)
                total_unique_pages_space += len(unique_pages)

                # Increment start_pages for next page of results
                start_pages += limit_pages

            # Display space name and total unique pages for current space
            print(f"Space: {space_name}, Total Unique Pages: {total_unique_pages_space}")

            # Update total_unique_pages with unique pages for current space
            total_unique_pages += total_unique_pages_space

        # Increment start_spaces for next batch of spaces
        start_spaces += limit_spaces

        # If there are more spaces to fetch, continue from the last space
        if 'next' in spaces_data.get('_links', {}):
            next_url = spaces_data['_links']['next']
            start_spaces = int(next_url.split('start=')[-1])

    total_unique_spaces = len(unique_spaces)
    print("\nTotal unique spaces in Confluence:", total_unique_spaces)
    print("Total unique pages in Confluence:", total_unique_pages)

    return {"total_unique_spaces": total_unique_spaces, "total_unique_pages": total_unique_pages}

get_total_unique_spaces_and_pages_in_confluence()



Total unique spaces in Confluence: 0
Total unique pages in Confluence: 0


{'total_unique_spaces': 0, 'total_unique_pages': 0}